In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import networkx as nx

import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.9.1  Device: cpu


In [2]:
batch_size = 32

train_dataset = datasets.MNIST('./data', 
                               train=True, 
                               download=True, 
                               transform=transforms.ToTensor())

validation_dataset = datasets.MNIST('./data', 
                                    train=False, 
                                    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False)

/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [4]:
class Net(nn.Module):
    def __init__(self, Net):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(50, 50)
        self.fc3_drop = nn.Dropout(0.2)
        self.fc4 = nn.Linear(50, 10)

# return list of all output of layers, name different variables for each layer (search for linear)
# only linear layer MLP for function, could add "weight mode" to change how weights are calculated
# will change for convolutional layers
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        x = F.relu(self.fc3(x))
        x = self.fc3_drop(x)
        return F.log_softmax(self.fc4(x), dim=1)

model = Net(Net).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()



print(model)

print(model.fc1.weight.size())
print(model.fc2.weight.size())
print(model.fc3.weight.size())
print(model.fc4.weight.size())

Net(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (fc3_drop): Dropout(p=0.2, inplace=False)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
)
torch.Size([50, 784])
torch.Size([50, 50])
torch.Size([50, 50])
torch.Size([10, 50])


In [5]:
def train(epoch, log_interval=200):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [6]:
def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

In [7]:
%%time
epochs = 10

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(lossv, accv)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304095
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.256305
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.144591
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.532280
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.120404
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.507188
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.849179
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.596258
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.856538
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.412227

Validation set: Average loss: 0.4620, Accuracy: 8724/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.758229
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.899044
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.514975
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.633631
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.455454
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.435614
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.377698
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.518919
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.37

In [20]:
#function for making graph from model
#layers = layers of model as list
#In the forward function, only out equals linear 1 and 2
#Two different variables, out1 = linear1 of 
def graph_from_model(layers):
    G = nx.DiGraph()
    #Create nodes of graph
    for layer_index in range(0,len(layers)):
        layer = layers[layer_index]
        for neuron_index in range(0,layer.weight.size()[1]):
            print("n"+str(layer_index)+"_"+str(neuron_index))
            G.add_node("n"+str(layer_index)+"_"+str(neuron_index))

        print("Layer "+str(layer_index)+" done")

    for neuron_index in range(0,layers[len(layers)-1].weight.size()[0]):
            print("n"+str(len(layers))+"_"+str(neuron_index))
            G.add_node("n"+str(len(layers))+"_"+str(neuron_index))
    print("Output layer done")

    #Create edges of graph
    for layer_index in range(0,len(layers)):
        layer = layers[layer_index]
        for x in range(0,layer.weight.size()[0]):
            for y in range(0,layer.weight.size()[1]):
                inNode = "n"+str(layer_index)+"_"+str(y)
                outNode = "n"+str(layer_index+1)+"_"+str(x)
                G.add_weighted_edges_from([(inNode, outNode, layer.weight[x][y].item())])
                print("Added edge from "+inNode+" to "+outNode+" with weight "+str(layer.weight[x][y].item()))
        print("Output layer edges done")

    return G

In [21]:
G = graph_from_model([model.fc1, model.fc2, model.fc3, model.fc4])

n0_0
n0_1
n0_2
n0_3
n0_4
n0_5
n0_6
n0_7
n0_8
n0_9
n0_10
n0_11
n0_12
n0_13
n0_14
n0_15
n0_16
n0_17
n0_18
n0_19
n0_20
n0_21
n0_22
n0_23
n0_24
n0_25
n0_26
n0_27
n0_28
n0_29
n0_30
n0_31
n0_32
n0_33
n0_34
n0_35
n0_36
n0_37
n0_38
n0_39
n0_40
n0_41
n0_42
n0_43
n0_44
n0_45
n0_46
n0_47
n0_48
n0_49
n0_50
n0_51
n0_52
n0_53
n0_54
n0_55
n0_56
n0_57
n0_58
n0_59
n0_60
n0_61
n0_62
n0_63
n0_64
n0_65
n0_66
n0_67
n0_68
n0_69
n0_70
n0_71
n0_72
n0_73
n0_74
n0_75
n0_76
n0_77
n0_78
n0_79
n0_80
n0_81
n0_82
n0_83
n0_84
n0_85
n0_86
n0_87
n0_88
n0_89
n0_90
n0_91
n0_92
n0_93
n0_94
n0_95
n0_96
n0_97
n0_98
n0_99
n0_100
n0_101
n0_102
n0_103
n0_104
n0_105
n0_106
n0_107
n0_108
n0_109
n0_110
n0_111
n0_112
n0_113
n0_114
n0_115
n0_116
n0_117
n0_118
n0_119
n0_120
n0_121
n0_122
n0_123
n0_124
n0_125
n0_126
n0_127
n0_128
n0_129
n0_130
n0_131
n0_132
n0_133
n0_134
n0_135
n0_136
n0_137
n0_138
n0_139
n0_140
n0_141
n0_142
n0_143
n0_144
n0_145
n0_146
n0_147
n0_148
n0_149
n0_150
n0_151
n0_152
n0_153
n0_154
n0_155
n0_156
n0_157
n0_1

Added edge from n0_722 to n1_0 with weight 0.033824000507593155
Added edge from n0_723 to n1_0 with weight 0.03525351732969284
Added edge from n0_724 to n1_0 with weight 0.02466924861073494
Added edge from n0_725 to n1_0 with weight 0.006341671571135521
Added edge from n0_726 to n1_0 with weight 0.024712081998586655
Added edge from n0_727 to n1_0 with weight -0.030758876353502274
Added edge from n0_728 to n1_0 with weight 0.013512004166841507
Added edge from n0_729 to n1_0 with weight -0.023136932402849197
Added edge from n0_730 to n1_0 with weight -0.03453065827488899
Added edge from n0_731 to n1_0 with weight 0.03183582425117493
Added edge from n0_732 to n1_0 with weight 0.02498725615441799
Added edge from n0_733 to n1_0 with weight -0.009046893566846848
Added edge from n0_734 to n1_0 with weight 0.016217047348618507
Added edge from n0_735 to n1_0 with weight -0.024446366354823112
Added edge from n0_736 to n1_0 with weight -0.03025689907371998
Added edge from n0_737 to n1_0 with weig

Added edge from n0_654 to n1_2 with weight 0.016568215563893318
Added edge from n0_655 to n1_2 with weight -0.018724901601672173
Added edge from n0_656 to n1_2 with weight -0.005565857049077749
Added edge from n0_657 to n1_2 with weight -0.006597988307476044
Added edge from n0_658 to n1_2 with weight 0.04030118137598038
Added edge from n0_659 to n1_2 with weight -0.009303128346800804
Added edge from n0_660 to n1_2 with weight -0.0333591029047966
Added edge from n0_661 to n1_2 with weight -0.007315328810364008
Added edge from n0_662 to n1_2 with weight -0.006073507014662027
Added edge from n0_663 to n1_2 with weight -0.03258083760738373
Added edge from n0_664 to n1_2 with weight -0.03770563751459122
Added edge from n0_665 to n1_2 with weight -0.05547277629375458
Added edge from n0_666 to n1_2 with weight 0.004853571765124798
Added edge from n0_667 to n1_2 with weight -0.01447894424200058
Added edge from n0_668 to n1_2 with weight -0.016053231433033943
Added edge from n0_669 to n1_2 with

Added edge from n0_585 to n1_4 with weight -0.029957463964819908
Added edge from n0_586 to n1_4 with weight 0.007517481688410044
Added edge from n0_587 to n1_4 with weight 0.011706547811627388
Added edge from n0_588 to n1_4 with weight 0.028732039034366608
Added edge from n0_589 to n1_4 with weight -0.014716311357915401
Added edge from n0_590 to n1_4 with weight -0.011623875238001347
Added edge from n0_591 to n1_4 with weight -0.031863581389188766
Added edge from n0_592 to n1_4 with weight -0.030131597071886063
Added edge from n0_593 to n1_4 with weight -0.05995182320475578
Added edge from n0_594 to n1_4 with weight 0.019428398460149765
Added edge from n0_595 to n1_4 with weight 0.02871684357523918
Added edge from n0_596 to n1_4 with weight 0.048642124980688095
Added edge from n0_597 to n1_4 with weight 0.03130495548248291
Added edge from n0_598 to n1_4 with weight 0.007145298644900322
Added edge from n0_599 to n1_4 with weight 0.04975740984082222
Added edge from n0_600 to n1_4 with we

Added edge from n0_517 to n1_6 with weight 0.022211970761418343
Added edge from n0_518 to n1_6 with weight 0.01861594244837761
Added edge from n0_519 to n1_6 with weight -0.009870795533061028
Added edge from n0_520 to n1_6 with weight 0.042581893503665924
Added edge from n0_521 to n1_6 with weight -0.003460842650383711
Added edge from n0_522 to n1_6 with weight 0.07342365384101868
Added edge from n0_523 to n1_6 with weight 0.049832750111818314
Added edge from n0_524 to n1_6 with weight 0.0373123399913311
Added edge from n0_525 to n1_6 with weight 0.024462468922138214
Added edge from n0_526 to n1_6 with weight 0.034286171197891235
Added edge from n0_527 to n1_6 with weight 0.007250161841511726
Added edge from n0_528 to n1_6 with weight 0.02096535824239254
Added edge from n0_529 to n1_6 with weight -0.021780017763376236
Added edge from n0_530 to n1_6 with weight -0.020360298454761505
Added edge from n0_531 to n1_6 with weight 0.006604216992855072
Added edge from n0_532 to n1_6 with weigh

...

Added edge from n0_110 to n1_48 with weight -0.016119202598929405
Added edge from n0_111 to n1_48 with weight 0.005991145968437195
Added edge from n0_112 to n1_48 with weight -0.023246532306075096
Added edge from n0_113 to n1_48 with weight -0.022128839045763016
Added edge from n0_114 to n1_48 with weight -0.002440770622342825
Added edge from n0_115 to n1_48 with weight 0.01739269122481346
Added edge from n0_116 to n1_48 with weight -0.02660061977803707
Added edge from n0_117 to n1_48 with weight -0.03603357821702957
Added edge from n0_118 to n1_48 with weight 0.0020060203969478607
Added edge from n0_119 to n1_48 with weight -0.034471649676561356
Added edge from n0_120 to n1_48 with weight 0.0034395905677229166
Added edge from n0_121 to n1_48 with weight -0.020552614703774452
Added edge from n0_122 to n1_48 with weight -0.08601660281419754
Added edge from n0_123 to n1_48 with weight -0.05048570781946182
Added edge from n0_124 to n1_48 with weight -0.09478198736906052
Added edge from n0

Added edge from n1_9 to n2_10 with weight 0.01824651099741459
Added edge from n1_10 to n2_10 with weight -0.057304855436086655
Added edge from n1_11 to n2_10 with weight 0.010824338532984257
Added edge from n1_12 to n2_10 with weight 0.06064590811729431
Added edge from n1_13 to n2_10 with weight -0.019580766558647156
Added edge from n1_14 to n2_10 with weight 0.11783767491579056
Added edge from n1_15 to n2_10 with weight -0.11656387150287628
Added edge from n1_16 to n2_10 with weight -0.08272039145231247
Added edge from n1_17 to n2_10 with weight -0.12919479608535767
Added edge from n1_18 to n2_10 with weight 0.0993097722530365
Added edge from n1_19 to n2_10 with weight 0.00559425400570035
Added edge from n1_20 to n2_10 with weight 0.07195550948381424
Added edge from n1_21 to n2_10 with weight -0.09148551523685455
Added edge from n1_22 to n2_10 with weight -0.04514859989285469
Added edge from n1_23 to n2_10 with weight 0.0005750283016823232
Added edge from n1_24 to n2_10 with weight -0

Added edge from n1_8 to n2_40 with weight 0.23884013295173645
Added edge from n1_9 to n2_40 with weight -0.003984867129474878
Added edge from n1_10 to n2_40 with weight -0.0030644808430224657
Added edge from n1_11 to n2_40 with weight -0.2006697803735733
Added edge from n1_12 to n2_40 with weight -0.0894288644194603
Added edge from n1_13 to n2_40 with weight -0.20954462885856628
Added edge from n1_14 to n2_40 with weight -0.382057249546051
Added edge from n1_15 to n2_40 with weight -0.04210108146071434
Added edge from n1_16 to n2_40 with weight -0.24090461432933807
Added edge from n1_17 to n2_40 with weight -0.06446336209774017
Added edge from n1_18 to n2_40 with weight -0.02142028510570526
Added edge from n1_19 to n2_40 with weight -0.05699425935745239
Added edge from n1_20 to n2_40 with weight 0.07531358301639557
Added edge from n1_21 to n2_40 with weight 0.24187296628952026
Added edge from n1_22 to n2_40 with weight -0.2895022928714752
Added edge from n1_23 to n2_40 with weight 0.01

Added edge from n2_7 to n3_20 with weight -0.057587988674640656
Added edge from n2_8 to n3_20 with weight 0.048235077410936356
Added edge from n2_9 to n3_20 with weight 0.21150904893875122
Added edge from n2_10 to n3_20 with weight 0.0954497829079628
Added edge from n2_11 to n3_20 with weight -0.021602923050522804
Added edge from n2_12 to n3_20 with weight 0.21583321690559387
Added edge from n2_13 to n3_20 with weight 0.23354358971118927
Added edge from n2_14 to n3_20 with weight 0.06630540639162064
Added edge from n2_15 to n3_20 with weight -0.0013448642566800117
Added edge from n2_16 to n3_20 with weight 0.08958124369382858
Added edge from n2_17 to n3_20 with weight 0.11061065644025803
Added edge from n2_18 to n3_20 with weight 0.07918404787778854
Added edge from n2_19 to n3_20 with weight -0.21343015134334564
Added edge from n2_20 to n3_20 with weight -0.008203888311982155
Added edge from n2_21 to n3_20 with weight 0.01847405545413494
Added edge from n2_22 to n3_20 with weight 0.238

Added edge from n3_5 to n4_0 with weight -0.26160281896591187
Added edge from n3_6 to n4_0 with weight 0.2530919909477234
Added edge from n3_7 to n4_0 with weight -0.14810395240783691
Added edge from n3_8 to n4_0 with weight -0.1184132769703865
Added edge from n3_9 to n4_0 with weight -0.09029360860586166
Added edge from n3_10 to n4_0 with weight 0.2958027422428131
Added edge from n3_11 to n4_0 with weight -0.03490859642624855
Added edge from n3_12 to n4_0 with weight 0.34693410992622375
Added edge from n3_13 to n4_0 with weight -0.2616521716117859
Added edge from n3_14 to n4_0 with weight 0.12430316209793091
Added edge from n3_15 to n4_0 with weight 0.36761367321014404
Added edge from n3_16 to n4_0 with weight -0.3315233588218689
Added edge from n3_17 to n4_0 with weight -0.388335645198822
Added edge from n3_18 to n4_0 with weight 0.318263441324234
Added edge from n3_19 to n4_0 with weight -0.12003298103809357
Added edge from n3_20 to n4_0 with weight 0.305474191904068
Added edge from